# 16. RNN과 어텐션을 사용한 자연어 처리

In [1]:
# 공통 모듈 임포트
import numpy as np
import os

# 깔금한 그래프 출력을 위해
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
plt.rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

## 16.1 Char-RNN으로 셰익스피어 문체 생성하기

### 16.1.1 훈련 데이터셋 만들기

In [ ]:
import tensorflow as tf

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

shakespeare_url = 'https://homl.info/shakespeare'
filepath = tf.keras.utils.get_file('shakespeare.txt', shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [ ]:
print(shakespeare_text[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


In [ ]:
text_vec_layer = tf.keras.layers.TextVectorization(split = 'character', standardize='lower')
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]

2023-11-01 17:02:46.485173: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [ ]:
encoded -= 2
n_tokens = text_vec_layer.vocabulary_size() - 2
dataset_size = len(encoded)

In [ ]:
def to_dataset(sequence, length, shuffle = False, seed = None, batch_size = 32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift = 1, drop_remainder = True)
    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))
    if shuffle:
        ds = ds.shuffle(buffer_size = 100_000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [ ]:
length = 100
tf.random.set_seed(42)
train_set = to_dataset(encoded[:1_000_000], length=length, shuffle=True, seed = 42)
valid_set = to_dataset(encoded[1_000_000:1_060_000], length=length)
test_set = to_dataset(encoded[1_060_000:], length=length)

### 16.1.2 Char-RNN 모델 만들고 훈련하기

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation = 'softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
model_ckpt = tf.keras.callbacks.ModelCheckpoint('my_shakespeare_model', monitor = 'val_accuracy', save_best_only=True)
history = model.fit(train_set, validation_data= valid_set, epochs=2, callbacks=[model_ckpt])

Epoch 1/2


2023-11-01 17:02:53.152280: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-11-01 17:03:01.229903: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-11-01 17:03:01.654143: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


KeyboardInterrupt: 

In [ ]:
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),
    model
])

In [ ]:
y_proba = shakespeare_model.predict(['To be or not to b'])[0, -1]
y_pred = tf.argmax(y_proba)
text_vec_layer.get_vocabulary()[y_pred + 2]

2023-10-30 02:07:06.127252: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-30 02:07:06.240412: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 645ms/step


'e'

### 16.1.3 가짜 셰익스피어 텍스트 생성하기

In [ ]:
def next_char(text, temperature = 1):
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logit = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logit, num_samples=1)[0, 0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [ ]:
def extend_text(text, n_chars = 50, temperature = 1):
    for _ in range(n_chars):
        text += next_char(text, temperature)

    return text

In [ ]:
tf.random.set_seed(42)

In [ ]:
extend_text('To be or not to be', temperature=0.01)

1/1 [==============================] - 0s 149ms/step


2023-10-30 02:07:07.032339: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-30 02:07:07.102589: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 38ms/step


'To be or not to be the the the the the the the the the the the the t'

In [ ]:
extend_text('To be or not to be', temperature=1)

1/1 [==============================] - 0s 30ms/step


'To be or not to bes\n\nibosie:\nberp, grecled,\na mance grrore me wilich'

In [ ]:
extend_text('To be or not to be', temperature=100)

1/1 [==============================] - 0s 32ms/step


"To be or not to beg ,mt'&o3f:ady-$\nws-nse?pws&ertj-vberdjw!c-yjewznq"

### 16.1.4 상태가 있는 RNN

## 16.2 감성 분석

In [ ]:
import tensorflow_datasets as tfds

raw_train_set, raw_valid_set, raw_test_set = tfds.load(
    name = 'imdb_reviews',
    split = ['train[:90%]', 'train[90%:]', 'test'],
    as_supervised=True
)

tf.random.set_seed(42)
train_set = raw_train_set.shuffle(5000, seed=42).batch(32).prefetch(1)
valid_set = raw_valid_set.batch(32).prefetch(1)
test_set = raw_test_set.batch(32).prefetch(1)

In [ ]:
for review, label in raw_train_set.take(4):
    print(review.numpy().decode('utf-8'))
    print('레이블:', label.numpy())

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.
레이블: 0
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development w

2023-10-30 14:02:39.954376: W tensorflow/core/kernels/data/cache_dataset_ops.cc:854] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [ ]:
vocab_size = 1000
text_vec_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size)
text_vec_layer.adapt(train_set.map(lambda reviews, labels: reviews))

2023-10-30 14:02:41.278466: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [ ]:
embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size, embed_size),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
history = model.fit(train_set, validation_data=valid_set, epochs=2)

Epoch 1/2


2023-10-30 14:02:43.788497: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-30 14:02:44.036392: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-30 14:02:44.205149: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


704/704 [==============================] - ETA: 0s - loss: 0.6940 - accuracy: 0.4978

2023-10-30 14:04:30.184217: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-30 14:04:30.317455: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


704/704 [==============================] - 113s 159ms/step - loss: 0.6940 - accuracy: 0.4978 - val_loss: 0.6929 - val_accuracy: 0.5012
Epoch 2/2
704/704 [==============================] - 141s 200ms/step - loss: 0.6928 - accuracy: 0.5053 - val_loss: 0.6952 - val_accuracy: 0.5012


### 16.2.1 마스킹

### 16.2.2 사전 훈련된 임베딩과 언어 모델 재사용하기

In [ ]:
import os
import tensorflow_hub as hub


os.environ['TFHUB_CACHE_DIR'] = 'my_tfhub_cache'
model = tf.keras.Sequential([
    hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4', trainable=True, dtype=tf.string, input_shape=[]),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
model.fit(train_set, validation_data=valid_set, epochs=10)

2023-10-30 14:08:26.220974: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Epoch 1/10


2023-10-30 14:08:34.643137: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


KeyboardInterrupt: 

## 16.3 신경망 기계 번역을 위한 인코더-디코더 네트워크

In [5]:
from pathlib import Path

url = 'https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip'
path = tf.keras.utils.get_file('spa-eng.zip', origin=url, cache_dir = 'datasets', extract=True)
text = (Path(path).with_name('spa-eng') / 'spa.txt').read_text()

In [6]:
text = text.replace('i', '').replace('¿', '')
pairs = [line.split('\t') for line in text.splitlines()]
np.random.shuffle(pairs)
sentences_en, sentences_es = zip(*pairs)

In [7]:
for i in range(3):
    print(sentences_en[i], '=>', sentences_en[i])

Leave the room. => Leave the room.
Tom's novel has been translated nto French. => Tom's novel has been translated nto French.
The attempt faled. => The attempt faled.


In [8]:
vocab_size = 1000
max_length = 50
text_vec_layer_en = tf.keras.layers.TextVectorization(vocab_size, output_sequence_length=max_length)
text_vec_layer_es = tf.keras.layers.TextVectorization(vocab_size, output_sequence_length=max_length)
text_vec_layer_en.adapt(sentences_en)
text_vec_layer_es.adapt([f'startofseq {s} endofseq' for s in sentences_es])

2023-11-02 17:15:21.189361: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-11-02 17:15:26.549975: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [9]:
text_vec_layer_en.get_vocabulary()[:10]

['', '[UNK]', 'the', 'i', 'to', 'you', 'tom', 'a', 's', 'he']

In [10]:
text_vec_layer_es.get_vocabulary()[:10]

['', '[UNK]', 'startofseq', 'endofseq', 'de', 'que', 'a', 'no', 'tom', 'la']

In [11]:
X_train = tf.constant(sentences_en[:100_000])
X_valid = tf.constant(sentences_en[100_000:])
X_train_dec = tf.constant([f'startofseq {s}' for s in sentences_es[:100_000]])
X_valid_dec = tf.constant([f'startofseq {s}' for s in sentences_es[100_000:]])
Y_train = text_vec_layer_es([f'{s} endofseq' for s in sentences_es[:100_000]])
Y_valid = text_vec_layer_es([f'{s} endofseq' for s in sentences_es[100_000:]])

In [12]:
encoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
decoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)

In [13]:
embed_size = 128
encoder_inputs_ids = text_vec_layer_en(encoder_inputs)
decoder_inputs_ids = text_vec_layer_es(decoder_inputs)
encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True)
decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True)
encoder_embeddings = encoder_embedding_layer(encoder_inputs_ids)
decoder_embeddings = decoder_embedding_layer(decoder_inputs_ids)

In [12]:
encoder = tf.keras.layers.LSTM(512, return_state=True)
encoder_outputs, *encoder_state = encoder(encoder_embeddings)

In [13]:
decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

In [14]:
output_layer = tf.keras.layers.Dense(vocab_size, activation='softmax')
Y_proba = output_layer(decoder_outputs)

In [ ]:
model = tf.keras.Model(inputs = [encoder_inputs, decoder_inputs], outputs = [Y_proba])
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
model.fit((X_train, X_train_dec), Y_train, epochs = 10, validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10


2023-10-31 16:46:58.923879: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-31 16:46:59.553081: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	for Tuple type infernce function 0
	while inferring type of node 'cond_40/output/_23'
2023-10-31 16:46:59.557679: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-31 16:4

3125/3125 [==============================] - ETA: 0s - loss: 3.5622 - accuracy: 0.3128

2023-10-31 16:51:18.478870: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-31 16:51:18.906063: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-31 16:51:19.107244: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3125/3125 [==============================] - 282s 89ms/step - loss: 3.5622 - accuracy: 0.3128 - val_loss: 3.2777 - val_accuracy: 0.3311
Epoch 2/10
3125/3125 [==============================] - 278s 89ms/step - loss: 3.1824 - accuracy: 0.3404 - val_loss: 3.1375 - val_accuracy: 0.3491
Epoch 3/10
3125/3125 [==============================] - 299s 96ms/step - loss: 3.0445 - accuracy: 0.3535 - val_loss: 3.0796 - val_accuracy: 0.3469
Epoch 4/10
3125/3125 [==============================] - 354s 113ms/step - loss: 2.9549 - accuracy: 0.3586 - val_loss: 3.0378 - val_accuracy: 0.3519
Epoch 5/10
3125/3125 [==============================] - 326s 104ms/step - loss: 2.8692 - accuracy: 0.3681 - val_loss: 3.0184 - val_accuracy: 0.3562
Epoch 6/10
3125/3125 [==============================] - 332s 106ms/step - loss: 2.7860 - accuracy: 0.3788 - val_loss: 2.9517 - val_accuracy: 0.3685
Epoch 7/10
3125/3125 [==============================] - 328s 105ms/step - loss: 2.6612 - accuracy: 0.4017 - val_loss: 2.9375 -

In [21]:
def translate(sentence_en):
    translation = ''
    for word_idx in range(max_length):
        X = np.array([sentence_en])
        X_dec = np.array(['startofseq ' + translation])
        y_proba = model.predict((X, X_dec))[0, word_idx]
        predicted_word_id = np.argmax(y_proba)
        predicted_word = text_vec_layer_es.get_vocabulary()[predicted_word_id]
        if predicted_word == 'endofseq':
            break
        translation += " " + predicted_word
    return translation.strip()


In [ ]:
translate("I like soccer")

2023-10-31 18:32:28.783861: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-31 18:32:29.368715: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-10-31 18:32:29.658688: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 0s 41ms/step


'tom no [UNK] [UNK]'

In [ ]:
translate("I like soccer and also going to the beach")

1/1 [==============================] - 0s 37ms/step


'tom no [UNK] [UNK]'

### 16.3.1 양방향 RNN

In [ ]:
encoder = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_state=True))

In [ ]:
encoder_outputs, *encoder_state = encoder(encoder_embeddings)
encoder_state = [tf.concat(encoder_state[::2], axis=-1),
                 tf.concat(encoder_state[1::2], axis=-1)]

### 16.3.2 빔 서치

## 16.4 어텐션 메커니즘

In [3]:
import tensorflow as tf

tf.random.set_seed(42)
encoder = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True, return_state=True))

2023-11-02 17:14:33.787042: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2023-11-02 17:14:33.787068: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2023-11-02 17:14:33.787076: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2023-11-02 17:14:33.787138: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-02 17:14:33.787170: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [16]:
encoder_outputs, *encoder_state = encoder(encoder_embeddings)
encoder_state = [tf.concat(encoder_state[::2], axis=-1),  # 단기 (0 & 2)
                 tf.concat(encoder_state[1::2], axis=-1)]  # 장기 (1 & 3)
decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

In [17]:
attention_layer = tf.keras.layers.Attention()
attention_outputs = attention_layer([decoder_outputs, encoder_outputs])

output_layer = tf.keras.layers.Dense(vocab_size, activation='softmax')
Y_proba = output_layer(attention_outputs)

In [19]:
model = tf.keras.Model(inputs = [encoder_inputs, decoder_inputs], outputs=[Y_proba])
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
model.fit((X_train, X_train_dec), Y_train, epochs=10, validation_data = ((X_valid, X_valid_dec), Y_valid))

Epoch 1/10
3125/3125 [==============================] - 142s 38ms/step - loss: 2.9484 - accuracy: 0.4420 - val_loss: 1.8391 - val_accuracy: 0.5964
Epoch 2/10
3125/3125 [==============================] - 97s 31ms/step - loss: 1.5628 - accuracy: 0.6446 - val_loss: 1.4477 - val_accuracy: 0.6654
Epoch 3/10
3125/3125 [==============================] - 94s 30ms/step - loss: 1.3003 - accuracy: 0.6900 - val_loss: 1.3296 - val_accuracy: 0.6869
Epoch 4/10
3125/3125 [==============================] - 98s 31ms/step - loss: 1.1583 - accuracy: 0.7159 - val_loss: 1.2838 - val_accuracy: 0.6959
Epoch 5/10
3125/3125 [==============================] - 99s 32ms/step - loss: 1.0536 - accuracy: 0.7356 - val_loss: 1.2593 - val_accuracy: 0.7039
Epoch 6/10
3125/3125 [==============================] - 94s 30ms/step - loss: 0.9661 - accuracy: 0.7528 - val_loss: 1.2580 - val_accuracy: 0.7058
Epoch 7/10
3125/3125 [==============================] - 97s 31ms/step - loss: 0.8916 - accuracy: 0.7682 - val_loss: 1.2697 

In [22]:
translate("I like soccer and also going to the beach")

1/1 [==============================] - 0s 29ms/step


'yo yo [UNK] fútbol y tambén [UNK] la playa'

In [23]:
translate("hello!")

1/1 [==============================] - 0s 27ms/step


'[UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK]'

### 16.4.1 트랜스포머 구조: 어텐션만 있으면 된다

In [25]:
max_length = 50
embed_size = 128
pos_embed_layer = tf.keras.layers.Embedding(max_length, embed_size)
batch_max_len_enc = tf.shape(encoder_embeddings)[1]
encoder_in = encoder_embeddings + pos_embed_layer(tf.range(batch_max_len_enc))
batch_max_len_dec = tf.shape(decoder_embeddings)[1]
decoder_in = decoder_embeddings + pos_embed_layer(tf.range(batch_max_len_dec))

In [26]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, max_length, embed_size, dtype=tf.float32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        assert embed_size % 2 == 0, "embed_size must be even"
        p, i = np.meshgrid(np.arange(max_length),
                           2 * np.arange(embed_size // 2))
        pos_emb = np.empty((1, max_length, embed_size))
        pos_emb[0, :, ::2] = np.sin(p / 10_000 ** (i / embed_size)).T
        pos_emb[0, :, 1::2] = np.cos(p / 10_000 ** (i / embed_size)).T
        self.pos_encodings = tf.constant(pos_emb.astype(self.dtype))
        self.supports_masking = True

    def call(self, inputs):
        batch_max_length = tf.shape(inputs)[1]
        return inputs + self.pos_encodings[:, :batch_max_length]

In [27]:
pos_embed_layer = PositionalEncoding(max_length, embed_size)
encoder_in = pos_embed_layer(encoder_embeddings)
decoder_in = pos_embed_layer(decoder_embeddings)

In [28]:
N = 2
num_heads = 8
dropout_rate = 0.1
n_units = 128
encoder_pad_mask = tf.math.not_equal(encoder_inputs_ids, 0)[:, tf.newaxis]
Z = encoder_in
for _ in range(N):
    skip = Z
    attn_layer = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    Z = attn_layer(Z, value=Z, attention_mask = encoder_pad_mask)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))
    skip = Z
    Z = tf.keras.layers.Dense(n_units, activation='relu')(Z)
    Z = tf.keras.layers.Dense(embed_size)(Z)
    Z = tf.keras.layers.Dropout(dropout_rate)(Z)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))

In [29]:
decoder_pad_mask = tf.math.not_equal(decoder_inputs_ids, 0)[:, tf.newaxis]
causal_mask = tf.linalg.band_part(tf.ones((batch_max_len_dec, batch_max_len_dec), tf.bool), -1, 0)

In [31]:
encoder_outputs = Z
Z = decoder_in
for _ in range(N):
    skip = Z
    attn_layer = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    Z = attn_layer(Z, value=Z, attention_mask = causal_mask & decoder_pad_mask)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))
    skip = Z
    attn_layer = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    Z = attn_layer(Z, value=encoder_outputs, attention_mask = encoder_pad_mask)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))
    skip = Z
    Z = tf.keras.layers.Dense(n_units, activation='relu')(Z)
    Z = tf.keras.layers.Dense(embed_size)(Z)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))

In [32]:
Y_proba = tf.keras.layers.Dense(vocab_size, activation='softmax')(Z)
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs = [Y_proba])
model.compile(loss='sparse_categorical_crossentropy', optimizer = 'nadam', metrics=['accuracy'])
model.fit((X_train, X_train_dec), Y_train, epochs = 10, validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10


2023-11-02 19:47:15.318360: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


   9/3125 [..............................] - ETA: 1:05:18 - loss: 5.8682 - accuracy: 0.1394

KeyboardInterrupt: 

## 16.5 언어 모델 분야의 최근 혁신

## 16.6 비전 트랜스포머

## 16.7 허깅 페이스의 트랜스포머스 라이브러리

In [2]:
from transformers import pipeline

classifier = pipeline('sentiment-analysis')
result = classifier('The actors were very convincing.')
result

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'POSITIVE', 'score': 0.9998071789741516}]

In [3]:
classifier(['I am from India.', 'I am from Iraq.'])

[{'label': 'POSITIVE', 'score': 0.9896161556243896},
 {'label': 'NEGATIVE', 'score': 0.9811071157455444}]

In [4]:
model_name = 'huggingface/distilbert-base-uncased-finetuned-mnli'
classifier_mnli = pipeline('text-classification', model=model_name)
classifier_mnli('She loves me. [SEP] She loves me not')

[{'label': 'contradiction', 'score': 0.6888487935066223}]

## 16.8 연습문제

### 8

In [3]:
import tensorflow as tf
import numpy as np

In [6]:
default_reber_grammar = [
    [("B", 1)],           # (state 0) =B=>(state 1)
    [("T", 2), ("P", 3)], # (state 1) =T=>(state 2) or =P=>(state 3)
    [("S", 2), ("X", 4)], # (state 2) =S=>(state 2) or =X=>(state 4)
    [("T", 3), ("V", 5)], # 등등 ...
    [("X", 3), ("S", 6)],
    [("P", 4), ("V", 6)],
    [("E", None)]]        # (state 6) =E=>(terminal state)

embedded_reber_grammar = [
    [("B", 1)],
    [("T", 2), ("P", 3)],
    [(default_reber_grammar, 4)],
    [(default_reber_grammar, 5)],
    [("T", 6)],
    [("P", 6)],
    [("E", None)]]

def generate_string(grammar):
    state = 0
    output = []
    while state is not None:
        index = np.random.randint(len(grammar[state]))
        production, state = grammar[state][index]
        if isinstance(production, list):
            production = generate_string(grammar=production)
        output.append(production)
    return "".join(output)

In [7]:
np.random.seed(42)

for _ in range(25):
    print(generate_string(default_reber_grammar), end = " ")

BTXXTTVPXTVPXTTVPSE BPVPSE BTXSE BPVVE BPVVE BTSXSE BPTVPXTTTVVE BPVVE BTXSE BTXXVPSE BPTTTTTTTTVVE BTXSE BPVPSE BTXSE BPTVPSE BTXXTVPSE BPVVE BPVVE BPVVE BPTTVVE BPVVE BPVVE BTXXVVE BTXXVVE BTXXVPXVVE 

In [9]:
np.random.seed(42)

for _ in range(25):
    print(generate_string(embedded_reber_grammar), end=" ")

BTBPTTTVPXTVPXTTVPSETE BPBPTVPSEPE BPBPVVEPE BPBPVPXVVEPE BPBTXXTTTTVVEPE BPBPVPSEPE BPBTXXVPSEPE BPBTSSSSSSSXSEPE BTBPVVETE BPBTXXVVEPE BPBTXXVPSEPE BTBTXXVVETE BPBPVVEPE BPBPVVEPE BPBTSXSEPE BPBPVVEPE BPBPTVPSEPE BPBTXXVVEPE BTBPTVPXVVETE BTBPVVETE BTBTSSSSSSSXXVVETE BPBTSSSXXTTTTVPSEPE BTBPTTVVETE BPBTXXTVVEPE BTBTXSETE 

In [34]:
POSSIBLE_CHARS = "BEPSTVX"

def generate_corrupted_string(grammar, chars=POSSIBLE_CHARS):
    good_string = generate_string(grammar)
    index = np.random.randint(len(good_string))
    good_char = good_string[index]
    bad_char = np.random.choice(sorted(set(chars) - set(good_char)))
    return good_string[:index] + bad_char + good_string[index + 1:]


In [35]:
np.random.seed(42)

for _ in range(25):
    print(generate_corrupted_string(embedded_reber_grammar), end=" ")

BTBPTTTPPXTVPXTTVPSETE BPBTXEEPE BPBPTVVVEPE BPBTSSSSXSETE BPTTXSEPE BTBPVPXTTTTTTEVETE BPBTXXSVEPE BSBPTTVPSETE BPBXVVEPE BEBTXSETE BPBPVPSXPE BTBPVVVETE BPBTSXSETE BPBPTTTPTTTTTVPSEPE BTBTXXTTSTVPSETE BBBTXSETE BPBTPXSEPE BPBPVPXTTTTVPXTVPXVPXTTTVVEVE BTBXXXTVPSETE BEBTSSSSSXXVPXTVVETE BTBXTTVVETE BPBTXSTPE BTBTXXTTTVPSBTE BTBTXSETX BTBTSXSSTE 

In [36]:
def string_to_ids(s, chars=POSSIBLE_CHARS):
    return [chars.index(c) for c in s]

In [37]:
string_to_ids("BTTTXXVVETE")

[0, 4, 4, 4, 6, 6, 5, 5, 1, 4, 1]

In [38]:
def generate_dataset(size):
    good_strings = [
        string_to_ids(generate_string(embedded_reber_grammar))
        for _ in range(size // 2)
    ]
    bad_strings = [
        string_to_ids(generate_corrupted_string(embedded_reber_grammar))
        for _ in range(size - size // 2)
    ]
    all_strings = good_strings + bad_strings
    X = tf.ragged.constant(all_strings, ragged_rank=1)
    y = np.array([[1.] for _ in range(len(good_strings))] +
                 [[0.] for _ in range(len(bad_strings))])
    return X, y

In [39]:
np.random.seed(42)

X_train, y_train = generate_dataset(10000)
X_valid, y_valid = generate_dataset(2000)

In [41]:
np.random.seed(42)
tf.random.set_seed(42)

embedding_size = 5

model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=[None], dtype=tf.int32, ragged=True),
    tf.keras.layers.Embedding(input_dim=len(POSSIBLE_CHARS),
                              output_dim=embedding_size),
    tf.keras.layers.GRU(30),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
optimizer = tf.keras.optimizers.SGD(learning_rate=0.02, momentum = 0.95,
                                    nesterov=True)
model.compile(loss="binary_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=20,
                    validation_data=(X_valid, y_valid))

Epoch 1/20


2023-11-05 17:06:07.574295: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node SGD/AssignVariableOp_1.


 13/313 [>.............................] - ETA: 43s - loss: 0.6934 - accuracy: 0.4736

KeyboardInterrupt: 

In [28]:
test_strings = ["BPBTSSSSSSSXXTTVPXVPXTTTTTVVETE",
                "BPBTSSSSSSSXXTTVPXVPXTTTTTVVEPE"]
X_test = tf.ragged.constant([string_to_ids(s) for s in test_strings], ragged_rank=1)

y_proba = model.predict(X_test)
print()
print("레버 문자열일 추정 확률:")
for index, string in enumerate(test_strings):
    print("{}: {:.2f}%".format(string, 100 * y_proba[index][0]))

1/1 [==============================] - 0s 70ms/step

레버 문자열일 추정 확률:
BPBTSSSSSSSXXTTVPXVPXTTTTTVVETE: nan%
BPBTSSSSSSSXXTTVPXVPXTTTTTVVEPE: nan%


### 9

In [4]:
from datetime import date

MONTHS = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

def random_dates(n_dates):
    min_date = date(1000, 1, 1).toordinal()
    max_date = date(9999, 12, 31).toordinal()
    ordinals = np.random.randint(max_date - min_date, size=n_dates) + min_date
    dates = [date.fromordinal(ordinal) for ordinal in ordinals]

    x = [MONTHS[dt.month - 1] + " " + dt.strftime("%d, %Y") for dt in dates]
    y = [dt.isoformat() for dt in dates]
    return x, y


In [15]:
np.random.seed(42)

n_dates = 3
x_example, y_example = random_dates(n_dates)
print("{:25s}{:25s}".format("Input", "Target"))
print("-" * 50)
for idx in range(n_dates):
    print("{:25s}{:25s}".format(x_example[idx], y_example[idx]))

Input                    Target                   
--------------------------------------------------
September 20, 7075       7075-09-20               
May 15, 8579             8579-05-15               
January 11, 7103         7103-01-11               


In [16]:
INPUT_CHARS = ''.join(sorted(set(''.join(MONTHS) + '0123456789, ')))
INPUT_CHARS

' ,0123456789ADFJMNOSabceghilmnoprstuvy'

In [17]:
OUTPUT_CHARS = '0123456789-'

In [21]:
def date_str_to_ids(date_str, chars=INPUT_CHARS):
    return [chars.index(c) for c in date_str]

In [22]:
date_str_to_ids(x_example[0], INPUT_CHARS)

[19, 23, 31, 34, 23, 28, 21, 23, 32, 0, 4, 2, 1, 0, 9, 2, 9, 7]

In [23]:
date_str_to_ids(y_example[0], OUTPUT_CHARS)

[7, 0, 7, 5, 10, 0, 9, 10, 2, 0]

In [26]:
def prepare_date_strs(date_strs, chars=INPUT_CHARS):
    X_ids = [date_str_to_ids(dt, chars) for dt in date_strs]
    X = tf.ragged.constant(X_ids, ragged_rank=1)
    return (X + 1).to_tensor()

def create_dataset(n_dates):
    x, y = random_dates(n_dates)
    return prepare_date_strs(x, INPUT_CHARS), prepare_date_strs(y, OUTPUT_CHARS)

In [27]:
np.random.seed(42)

X_train, Y_train = create_dataset(10000)
X_valid, Y_valid = create_dataset(2000)
X_test, Y_test = create_dataset(2000)

In [28]:
Y_train[0]

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([ 8,  1,  8,  6, 11,  1, 10, 11,  3,  1], dtype=int32)>

In [32]:
embedding_size = 32
max_output_length = Y_train.shape[1]

np.random.seed(42)
tf.random.set_seed(42)

encoder = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim = len(INPUT_CHARS) + 1, output_dim = embedding_size, input_shape = [None]),
    tf.keras.layers.LSTM(128)
])

decoder = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, return_sequences=True),
    tf.keras.layers.Dense(len(OUTPUT_CHARS) + 1, activation='softmax')
])

model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.RepeatVector(max_output_length),
    decoder
])

optimizer = tf.keras.optimizers.Nadam()
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X_train, Y_train, epochs=20, validation_data=(X_valid, Y_valid))

Epoch 1/20


2023-11-06 17:04:00.111511: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 15s 36ms/step - loss: 1.7095 - accuracy: 0.3825 - val_loss: 1.3277 - val_accuracy: 0.5161
Epoch 2/20
313/313 [==============================] - 10s 32ms/step - loss: 1.1621 - accuracy: 0.5811 - val_loss: 0.9958 - val_accuracy: 0.6339
Epoch 3/20
313/313 [==============================] - 10s 32ms/step - loss: 0.8306 - accuracy: 0.6944 - val_loss: 0.6976 - val_accuracy: 0.7361
Epoch 4/20
313/313 [==============================] - 10s 33ms/step - loss: 0.6029 - accuracy: 0.7591 - val_loss: 0.5310 - val_accuracy: 0.7819
Epoch 5/20
313/313 [==============================] - 11s 35ms/step - loss: 0.6524 - accuracy: 0.7568 - val_loss: 0.4415 - val_accuracy: 0.8218
Epoch 6/20
313/313 [==============================] - 11s 34ms/step - loss: 0.3517 - accuracy: 0.8597 - val_loss: 0.2902 - val_accuracy: 0.8862
Epoch 7/20
313/313 [==============================] - 11s 36ms/step - loss: 0.2383 - accuracy: 0.9137 - val_loss: 0.2001 - val_accuracy: 0.9334
Epo

In [33]:
def ids_to_date_strs(ids, chars=OUTPUT_CHARS):
    return ["".join([('?' + chars)[index] for index in sequence]) for sequence in ids]

In [34]:
X_new = prepare_date_strs(['September 17, 2009', 'July 14, 1789'])

In [35]:
ids = model.predict(X_new).argmax(axis=-1)
for date_str in ids_to_date_strs(ids):
    print(date_str)

1/1 [==============================] - 1s 563ms/step
2009-09-17
1789-07-14


In [36]:
X_new = prepare_date_strs(["May 02, 2020", "July 14, 1789"])
ids = model.predict(X_new).argmax(axis=-1)
for date_str in ids_to_date_strs(ids):
    print(date_str)

1/1 [==============================] - 0s 483ms/step
2020-02-02
1789-02-14


In [37]:
max_input_length = X_train.shape[1]

def prepare_date_strs_padded(date_strs):
    X = prepare_date_strs(date_strs)
    if X.shape[1] < max_input_length:
        X = tf.pad(X, [[0, 0], [0, max_input_length - X.shape[1]]])
    return X

def convert_date_strs(date_strs):
    X = prepare_date_strs_padded(date_strs)
    ids = model.predict(X).argmax(axis=-1)
    return ids_to_date_strs(ids)

In [38]:
convert_date_strs(["May 02, 2020", "July 14, 1789"])

1/1 [==============================] - 0s 44ms/step


['2020-05-02', '1789-07-14']

### 11

In [39]:
from transformers import TFOpenAIGPTLMHeadModel

model = TFOpenAIGPTLMHeadModel.from_pretrained('openai-gpt')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFOpenAIGPTLMHeadModel: ['h.5.attn.bias', 'h.2.attn.bias', 'h.11.attn.bias', 'h.9.attn.bias', 'h.4.attn.bias', 'h.1.attn.bias', 'h.6.attn.bias', 'h.3.attn.bias', 'h.10.attn.bias', 'h.8.attn.bias', 'h.7.attn.bias', 'h.0.attn.bias']
- This IS expected if you are initializing TFOpenAIGPTLMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFOpenAIGPTLMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFOpenAIGPTLMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFOpenAIGPTLMHeadModel for predicti

In [40]:
from transformers import OpenAIGPTTokenizer

tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [41]:
tokenizer('hello everyone')

{'input_ids': [3570, 1473], 'attention_mask': [1, 1]}

In [42]:
prompt_text = 'This royal throne of kings, this sceptred isle'
encoded_prompt = tokenizer.encode(prompt_text, add_special_tokens=False, return_tensors='tf')
encoded_prompt

<tf.Tensor: shape=(1, 10), dtype=int32, numpy=
array([[  616,  5751,  6404,   498,  9606,   240,   616, 26271,  7428,
        16187]], dtype=int32)>

In [43]:
num_sequences = 5
length = 40

generated_sequences = model.generate(
    input_ids = encoded_prompt, 
    do_sample = True,
    max_length = length + len(encoded_prompt[0]),
    temperature = 1.0,
    top_k=0,
    top_p=0.9,
    repetition_penalty=1.0,
    num_return_sequences = num_sequences
)

generated_sequences

<tf.Tensor: shape=(5, 50), dtype=int32, numpy=
array([[  616,  5751,  6404,   498,  9606,   240,   616, 26271,  7428,
        16187,   239, 40477,   556,   524,  1724,  4338,   504,   481,
          831,  1520,   546,   535, 12003,  4374,   240, 10341,   535,
         2236,  1404, 21760,   239,   998,   524,  2170,  1063,  1098,
          833,   604, 21617,   575,   240,   524,  9093,   626,   595,
         4203,  1129,   239,   487,  1787],
       [  616,  5751,  6404,   498,  9606,   240,   616, 26271,  7428,
        16187,   636,  1300,   666,   246, 12065,  1276,   240,   488,
          557,   622, 14404,  1546,  1260,   481, 19995,   240, 14404,
        30599,   636,  3226,   239,   500,   616,   638,   507,   509,
         1816,   525,  1007,  1594,   636,  1443,   580,  1632,   240,
          481,  6391,  7876,   498,   481],
       [  616,  5751,  6404,   498,  9606,   240,   616, 26271,  7428,
        16187,   239,   645,   512,   640,   246,  1250,   498,   547,
         3766

In [44]:
for sequence in generated_sequences:
    text = tokenizer.decode(sequence, clean_up_tokenization_spaces=True)
    print(text)
    print("-" * 80)

this royal throne of kings, this sceptred isle. 
 with his gaze fixed on the marquise's fearful features, domen's plan finally yielded. though his foot might once again have steeled him, his resolve did not dare work. he continued
--------------------------------------------------------------------------------
this royal throne of kings, this sceptred isle would turn into a hostile world, and as our wretched ship walked the shores, wretched alliances would begin. in this way it was known that another land would soon be lost, the twin cities of the
--------------------------------------------------------------------------------
this royal throne of kings, this sceptred isle. if you are a part of my court and not my friends, then consider your words.'
'i have no interest in governing a kingdom that craves nothing but to rule,'he
--------------------------------------------------------------------------------
this royal throne of kings, this sceptred isle, is the fifth king of mallorea, t